# _**Movie Recommender System**_ #

In [89]:
import json
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
import warnings
from surprise import Reader, Dataset, KNNBasic, SVD
import pickle
import random
from sklearn.utils import resample
from sklearn.feature_extraction.text import CountVectorizer
warnings.filterwarnings('ignore')

np.random.seed(42)
random.seed(42)

In [90]:
movies = pd.read_csv("../data/final data/movies_metadata.csv")
ratings = pd.read_csv("../data/final data/ratings_small.csv")
links = pd.read_csv("../data/final data/links_small.csv")
credits = pd.read_csv("../data/final data/credits.csv")
keywords = pd.read_csv("../data/final data/keywords.csv")

In [91]:
print("Shapes:")
print("movies:", movies.shape)
print("ratings:", ratings.shape)
print("links:", links.shape)
print("keywords:", keywords.shape)
print("credits:", credits.shape)

Shapes:
movies: (45466, 28)
ratings: (100004, 4)
links: (9125, 3)
keywords: (46419, 3)
credits: (45476, 14)


In [92]:
print("movies columns:", movies.columns.tolist())
print("ratings columns:", ratings.columns.tolist())
print("links columns:", links.columns.tolist())
print("keywords columns:", keywords.columns.tolist())
print("credits columns:", credits.columns.tolist())

movies columns: ['adult', 'budget', 'id', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title', 'video', 'vote_average', 'vote_count', 'name_belongs_to_collection', 'id_belongs_to_collection', 'poster_path_belongs_to_collection', 'backdrop_path_belongs_to_collection', 'name_genres', 'id_genres', 'name_production_countries', 'iso_3166_1_production_countries', 'name_production_companies', 'id_production_companies']
ratings columns: ['userId', 'movieId', 'rating', 'timestamp']
links columns: ['movieId', 'imdbId', 'tmdbId']
keywords columns: ['id', 'name_keywords', 'id_keywords']
credits columns: ['id', 'name_crew', 'department_crew', 'gender_crew', 'job_crew', 'profile_path_crew', 'id_crew', 'name_cast', 'order_cast', 'gender_cast', 'credit_id_cast', 'profile_path_cast', 'id_cast', 'character_cast']


In [93]:
links = links.dropna(subset=["tmdbId", "movieId"])
links["tmdbId"] = pd.to_numeric(links["tmdbId"], errors="coerce")
links = links.dropna(subset=["tmdbId"]).astype({"tmdbId": int, "movieId": int})
ml_to_tmdb = dict(zip(links["movieId"], links["tmdbId"]))

In [94]:
print("original ratings rows:", len(ratings))
ratings = ratings[ratings["movieId"].isin(ml_to_tmdb.keys())].copy()
ratings["tmdbId"] = ratings["movieId"].map(ml_to_tmdb)
print("ratings after mapping:", len(ratings))

original ratings rows: 100004
ratings after mapping: 99933


In [95]:
ratings["timestamp"] = pd.to_datetime(ratings["timestamp"], unit="s")
ratings = ratings.sort_values(["userId", "timestamp"])
last_idx = ratings.groupby("userId")["timestamp"].idxmax()
test_df = ratings.loc[last_idx].copy()
train_df = ratings.drop(index=last_idx).copy()

In [96]:
min_train_interactions = 3
valid_users = train_df.groupby("userId").size()
valid_users = valid_users[valid_users >= min_train_interactions].index
train_df = train_df[train_df["userId"].isin(valid_users)]
test_df = test_df[test_df["userId"].isin(valid_users)]

In [97]:
print("train users:", train_df["userId"].nunique())
print("test rows:", len(test_df))

train users: 671
test rows: 671


WR (popularity baseline)

In [98]:
for col in ["vote_count", "vote_average", "id", "title"]:
    if col not in movies.columns:
        raise ValueError(f"movies missing column {col}")

movies_small = movies[["id", "title", "vote_count", "vote_average", "popularity"]].copy()
movies_small["vote_count"] = pd.to_numeric(movies_small["vote_count"], errors="coerce").fillna(0)
movies_small["vote_average"] = pd.to_numeric(movies_small["vote_average"], errors="coerce").fillna(0)

C = movies_small["vote_average"].mean()
m = movies_small["vote_count"].quantile(0.80)

def imdb_weighted_rating(row, m=m, C=C):
    v = row["vote_count"]
    R = row["vote_average"]
    return (v / (v + m)) * R + (m / (v + m)) * C if (v + m) > 0 else 0.0

movies_small["wr"] = movies_small.apply(imdb_weighted_rating, axis=1)

movies_small["id"] = pd.to_numeric(movies_small["id"], errors="coerce")

movies_small = movies_small.dropna(subset=["id"])

movies_small["id"] = movies_small["id"].astype(int)

In [99]:
candidate_tmdb = set(ml_to_tmdb.values())
pop_candidates = movies_small[movies_small["id"].isin(candidate_tmdb)].copy()
pop_candidates = pop_candidates.sort_values("wr", ascending=False)
pop_ranked_tmdb = pop_candidates["id"].tolist()
print("Candidates for popularity:", len(pop_ranked_tmdb))

Candidates for popularity: 9099


Content-Based Recommender (CB)

In [100]:
def to_tokens(s):
    if pd.isna(s) or s == "":
        return []
    return [t.strip().replace(" ", "_") for t in str(s).split(",") if t.strip()]

for df in [movies, keywords, credits]:
    df["id"] = pd.to_numeric(df["id"], errors="coerce")
    df.dropna(subset=["id"], inplace=True)
    df["id"] = df["id"].astype(int)

In [101]:
kw_map = keywords[keywords["id"].isin(candidate_tmdb)][["id", "name_keywords"]].set_index("id")["name_keywords"].to_dict()

cast_map = credits[credits["id"].isin(candidate_tmdb)][["id", "name_cast"]].set_index("id")["name_cast"].to_dict()

crew_map = credits[credits["id"].isin(candidate_tmdb)][["id", "name_crew"]].set_index("id")["name_crew"].to_dict()

movies_cb = movies[movies["id"].isin(candidate_tmdb)][["id","overview","tagline","name_genres","title"]].copy()

In [102]:
def build_item_text(row, kw_map=kw_map, cast_map=cast_map, crew_map=crew_map):
    parts = []
    if isinstance(row.get("overview"), str): parts.append(row["overview"])
    if isinstance(row.get("tagline"), str): parts.append(row["tagline"])
    # genres
    for tok in to_tokens(row.get("name_genres")):
        parts.append(f"GENRE_{tok}")
    # keywords
    kws = kw_map.get(row["id"])
    if isinstance(kws, str):
        for tok in to_tokens(kws):
            parts.append(f"KW_{tok}")
    # cast
    cast = cast_map.get(row["id"])
    if isinstance(cast, str):
        for tok in to_tokens(cast)[:10]:
            parts.append(f"CAST_{tok}")
    # crew
    crew = crew_map.get(row["id"])
    if isinstance(crew, str):
        for tok in to_tokens(crew)[:10]:
            parts.append(f"CREW_{tok}")
    return " ".join(parts)

In [103]:
movies_cb["cb_text"] = movies_cb.apply(build_item_text, axis=1)
print("CB items:", len(movies_cb))

CB items: 9099


TF-IDF

In [104]:
vectorizer = TfidfVectorizer(max_features=50000, min_df=2, ngram_range=(1,2))
X_items = vectorizer.fit_transform(movies_cb["cb_text"])

item_index = {tmdb_id: idx for idx, tmdb_id in enumerate(movies_cb["id"].tolist())}
index_item = {idx: tmdb_id for tmdb_id, idx in item_index.items()}

print("X_items shape:", X_items.shape)

X_items shape: (9099, 50000)


User profiles(cb)

In [105]:
train_tmdb = train_df[train_df["tmdbId"].isin(item_index.keys())].copy()
train_tmdb["item_idx"] = train_tmdb["tmdbId"].map(item_index)

user_ids = sorted(train_tmdb["userId"].unique())
uid_to_index = {u:i for i,u in enumerate(user_ids)}


train_user_mean = train_df.groupby("userId")["rating"].mean().to_dict()
train_tmdb["w"] = train_tmdb.apply(lambda r: r["rating"] - train_user_mean[r["userId"]], axis=1)

rows = train_tmdb["item_idx"].astype(int).to_numpy()
cols = train_tmdb["userId"].map(uid_to_index).astype(int).to_numpy()
data = train_tmdb["w"].astype(np.float32).to_numpy()

M = sparse.coo_matrix((data, (rows, cols)), shape=(X_items.shape[0], len(uid_to_index))).tocsr()

user_cb_profiles = M.T.dot(X_items)
user_cb_profiles = normalize(user_cb_profiles.tocsr(), norm="l2", axis=1)

print("user_cb_profiles shape:", user_cb_profiles.shape)

user_cb_profiles shape: (671, 50000)


In [106]:
def build_user_profile(uid, ratings_df, item_index, X_items):
    """Build user profile as weighted mean of seen item vectors"""
    if uid not in ratings_df["userId"].values:
        return None
    rows = []
    weights = []
    user_mean = ratings_df[ratings_df["userId"]==uid]["rating"].mean()
    for _, row in ratings_df[ratings_df["userId"]==uid].iterrows():
        tmdb = row["tmdbId"]
        if tmdb not in item_index: continue
        idx = item_index[tmdb]
        rows.append(X_items[idx])
        weights.append(row["rating"] - user_mean)
    if not rows:
        return None
    mat = np.vstack([r.toarray() for r in rows])
    w = np.array(weights).reshape(-1,1)
    prof = (mat * w).mean(axis=0)
    return prof

In [107]:
def explain_recommendation(movie_id, user_seen_ids, movies_meta, kw_map, cast_map):
    """Generate short natural explanation (genre/actor overlap)."""
    explanation = []
    movie_genres = set(to_tokens(movies_meta.loc[movies_meta["id"]==movie_id,"name_genres"].values[0]))
    movie_cast = set(to_tokens(cast_map.get(movie_id,"")))
    # compare with seen movies
    seen_genres = set()
    seen_cast = set()
    for sid in user_seen_ids:
        if sid in movies_meta["id"].values:
            seen_genres |= set(to_tokens(movies_meta.loc[movies_meta["id"]==sid,"name_genres"].values[0]))
            seen_cast |= set(to_tokens(cast_map.get(sid,"")))
    g_common = movie_genres & seen_genres
    c_common = movie_cast & seen_cast
    if g_common:
        explanation.append(f"genre common: {list(g_common)[0]}")
    elif c_common:
        explanation.append(f"cast common: {list(c_common)[0]}")
    else:
        explanation.append("explanation")
    return "، ".join(explanation)

Recommender function(CB)

In [108]:
def recommend_popular(k, exclude_set=None):
    rec = []
    for tmdb in pop_ranked_tmdb:
        if exclude_set and tmdb in exclude_set:
            continue
        rec.append(tmdb)
        if len(rec) >= k:
            break
    return rec

In [109]:
def recommend_cb(user_id, k=10, exclude_set=None, min_score=0.0, cold_start_prefs=None):
    # cold-start
    if user_id not in uid_to_index:
        if cold_start_prefs and isinstance(cold_start_prefs,str):
            # make synthetic profile
            synthetic_vec = vectorizer.transform([cold_start_prefs])
            sims = (synthetic_vec @ X_items.T).toarray().ravel()
            order = np.argsort(-sims)
            rec = []
            for iidx in order:
                tmdb = index_item.get(iidx)
                if tmdb is None: continue
                if exclude_set and tmdb in exclude_set: continue
                rec.append((tmdb, sims[iidx]))
                if len(rec) >= k: break
            return [t for t, s in rec]
        else:
            # fallback to popularity
            return recommend_popular(k, exclude_set)

    # normal CB
    u_idx = uid_to_index[user_id]
    uvec = user_cb_profiles[u_idx, :]
    if uvec.nnz == 0:
        return recommend_popular(k, exclude_set)

    sims = (uvec @ X_items.T).toarray().ravel()
    order = np.argsort(-sims)
    rec = []
    for iidx in order:
        tmdb = index_item.get(iidx)
        if tmdb is None: continue
        if exclude_set and tmdb in exclude_set: continue
        if sims[iidx] >= min_score:
            rec.append((tmdb, sims[iidx]))
        if len(rec) >= k: break
    return [t for t, s in rec]

Collaborative Filtering

In [110]:
n_items = X_items.shape[0]

In [111]:
df_sur = train_tmdb[["userId","item_idx","rating"]].copy()
df_sur["userID"] = df_sur["userId"].map(uid_to_index).astype(str)
df_sur["itemID"] = df_sur["item_idx"].astype(str)

min_r = float(df_sur["rating"].min())
max_r = float(df_sur["rating"].max())
reader = Reader(rating_scale=(min_r, max_r))
data = Dataset.load_from_df(df_sur[["userID","itemID","rating"]], reader)

In [112]:
def _minmax(v):
    v = np.asarray(v, dtype=np.float32)
    if v.size == 0: return v
    vmin, vmax = float(np.min(v)), float(np.max(v))
    if not np.isfinite(vmin) or vmax - vmin < 1e-9:
        return np.zeros_like(v)
    return (v - vmin) / (vmax - vmin)

similarity

In [113]:
sim_options = {
    'name': 'pearson',
    'user_based': False
}
algo_sim = KNNBasic(k=100, min_k=5, sim_options=sim_options)
trainset = data.build_full_trainset()
algo_sim.fit(trainset)

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [114]:
def cf_item_item_scores(user_id):
    scores = np.zeros(n_items, dtype=np.float32)
    if user_id not in uid_to_index:
        return scores
    u_idx = uid_to_index[user_id]
    for i in range(n_items):
        scores[i] = algo_sim.predict(str(u_idx), str(i)).est
    return _minmax(scores)

Matrix Factorization

In [115]:
algo_mf = SVD(n_factors=80, n_epochs=25, lr_all=0.007, reg_all=0.05)
trainset = data.build_full_trainset()
algo_mf.fit(trainset)

In [116]:
def mf_scores_for_user(user_id):
    scores = np.zeros(n_items, dtype=np.float32)
    if user_id not in uid_to_index:
        return scores
    u_idx = uid_to_index[user_id]
    for i in range(n_items):
        scores[i] = algo_mf.predict(str(u_idx), str(i)).est
    return _minmax(scores)

In [117]:
def recommend_mf(user_id, k=10, exclude_set=None, model=algo_mf):
    scores = mf_scores_for_user(user_id)
    order = np.argsort(-scores)
    rec = [
        index_item[iidx]
        for iidx in order
        if index_item.get(iidx) is not None
        and (exclude_set is None or index_item[iidx] not in exclude_set)
    ]
    return rec[:k]

Hybrid

In [118]:
def cf_scores(user_id, method="item-item"):
    if method == "item-item":
        return cf_item_item_scores(user_id)
    elif method == "mf":
        return mf_scores_for_user(user_id)
    else:
        raise ValueError("method not supported")

In [119]:
def recommend_hybrid(user_id, k=10, exclude_set=None, alpha=0.3, cf_method="item-item"):
    # CB scores
    if user_id in uid_to_index:
        u_idx = uid_to_index[user_id]
        cb_scores = (user_cb_profiles[u_idx, :] @ X_items.T).toarray().ravel()
    else:
        cb_scores = None

    # CF scores
    try:
        cf_s = cf_scores(user_id, method=cf_method)
    except Exception:
        cf_s = None

    if cb_scores is None and cf_s is None:
        return recommend_popular(k, exclude_set)

    # normalize available signals to [0,1]
    def norm(v):
        if v is None or np.all(np.isclose(v, v[0] if len(v) > 0 else 0)):
            return None
        v = np.array(v, dtype=np.float32)
        return (v - v.min()) / (v.max() - v.min() + 1e-9)

    cbn = norm(cb_scores)
    cfn = norm(cf_s)

    if cbn is None and cfn is not None:
        s = cfn
    elif cfn is None and cbn is not None:
        s = cbn
    else:
        s = alpha * cfn + (1 - alpha) * cbn

    order = np.argsort(-s)
    rec = []
    for iidx in order:
        tmdb = index_item.get(int(iidx))
        if tmdb is None:
            continue
        if exclude_set and tmdb in exclude_set:
            continue
        rec.append(tmdb)
        if len(rec) >= k:
            break

    if len(rec) < k:
        need = k - len(rec)
        rec += [x for x in recommend_popular(k + need, exclude_set) if x not in rec][:need]

    return rec

evaluation

In [120]:
def precision_at_k(actual, pred, k):
    if k == 0: return 0.0
    return len(set(actual) & set(pred[:k])) / k

def recall_at_k(actual, pred, k):
    if len(actual) == 0: return 0.0
    return len(set(actual) & set(pred[:k])) / len(set(actual))

def ndcg_at_k(actual, pred, k):
    actual = set(actual)
    rel = [1.0 if p in actual else 0.0 for p in pred[:k]]
    dcg = sum([rel[i] / np.log2(i + 2) for i in range(len(rel))])
    idcg = sum([1.0 / np.log2(i + 2) for i in range(min(len(actual), k))])
    return (dcg / idcg) if idcg > 0 else 0.0

def hit_rate_at_k(actual, pred, k):
    return 1.0 if len(set(actual) & set(pred[:k])) > 0 else 0.0

def map_at_k(actual, pred, k):
    actual = set(actual)
    if len(actual) == 0: return 0.0
    hits = 0
    score = 0.0
    for i, p in enumerate(pred[:k], start=1):
        if p in actual:
            hits += 1
            score += hits / i
    return score / min(len(actual), k)

In [121]:
train_parts = []
test_parts = []

for uid, group in ratings.groupby("userId"):
    group = group.sort_values("timestamp")
    n = len(group)
    if n < 5:
        train_parts.append(group)
        continue
    cutoff = int(n * 0.8)
    train_parts.append(group.iloc[:cutoff])
    test_parts.append(group.iloc[cutoff:])

train_df = pd.concat(train_parts)
test_df  = pd.concat(test_parts)

In [122]:
train_seen = (
    train_df.groupby("userId")["tmdbId"]
    .apply(lambda s: set(int(x) for x in s.dropna().tolist()))
    .to_dict()
)

test_truth = (
    test_df.groupby("userId")["tmdbId"]
    .apply(lambda s: [int(x) for x in s.dropna().tolist()])
    .to_dict()
)

test_users = sorted(test_truth.keys())

In [123]:
def preds_cb(uid, k):
    excl = train_seen.get(uid, set())
    try:
        rec = recommend_cb(uid, k=k, exclude_set=excl)
    except Exception:
        rec = []
    return rec[:k]

def preds_cf_mf(uid, k):
    try:
        s = cf_scores(uid, method="mf")
    except Exception:
        return []
    order = np.argsort(-s)
    out = []
    excl = train_seen.get(uid, set())
    for iidx in order:
        mid = index_item.get(int(iidx))
        if mid is None:
            continue
        if mid in excl:
            continue
        out.append(mid)
        if len(out) >= k:
            break
    return out

def preds_hybrid(uid, k, alpha=0.3, cf_method="mf"):
    excl = train_seen.get(uid, set())
    try:
        rec = recommend_hybrid(uid, k=k, exclude_set=excl, alpha=alpha, cf_method=cf_method)
    except Exception:
        rec = []
    return rec[:k]

In [124]:
k_list = [10, 20]
models = {
    "CB": lambda uid, k: preds_cb(uid, k),
    "CF_MF": lambda uid, k: preds_cf_mf(uid, k),
    "HYBRID": lambda uid, k: preds_hybrid(uid, k, alpha=0.3, cf_method="mf"),
}

if "algo_sim" in globals():
    def preds_cf_item(uid, k):
        try:
            s = cf_scores(uid, method="item-item")
        except Exception:
            return []
        order = np.argsort(-s)
        out, excl = [], train_seen.get(uid, set())
        for iidx in order:
            mid = index_item.get(int(iidx))
            if mid is None or mid in excl:
                continue
            out.append(mid)
            if len(out) >= k:
                break
        return out
    models["CF_ITEM"] = preds_cf_item

In [125]:
def bootstrap_ci(values, n_boot=1000, alpha=0.05):
    if len(values) == 0: return (np.nan, np.nan)
    means = []
    values = np.array(values)
    n = len(values)
    for _ in range(n_boot):
        sample = resample(values, n_samples=n, replace=True)
        means.append(np.mean(sample))
    return np.percentile(means, 100 * (alpha/2)), np.percentile(means, 100 * (1-alpha/2))

In [126]:
def run_evaluation(models, k_list, metrics, test_users, test_truth, train_seen):
    per_user_metrics = {name: {f"{m}@{k}": [] for k in k_list for m in metrics}
                        for name in models.keys()}
    for name, fn in models.items():
        for k in k_list:
            for uid in test_users:
                actual = test_truth.get(uid, [])
                pred = fn(uid, k=k) or []
                p = precision_at_k(actual, pred, k)
                r = recall_at_k(actual, pred, k)
                n = ndcg_at_k(actual, pred, k)
                h = hit_rate_at_k(actual, pred, k)
                m = map_at_k(actual, pred, k)
                per_user_metrics[name][f"P@{k}"].append(p)
                per_user_metrics[name][f"R@{k}"].append(r)
                per_user_metrics[name][f"NDCG@{k}"].append(n)
                per_user_metrics[name][f"HR@{k}"].append(h)
                per_user_metrics[name][f"MAP@{k}"].append(m)
    summary_rows = []
    for name in models.keys():
        for k in k_list:
            row = {"model": name, "k": k}
            for m in metrics:
                vals = per_user_metrics[name][f"{m}@{k}"]
                row[m] = np.mean(vals)
                lo, hi = bootstrap_ci(vals, n_boot=500)
                row[f"{m}_ci95"] = f"[{lo:.3f}, {hi:.3f}]"
            summary_rows.append(row)
    return pd.DataFrame(summary_rows).sort_values(["k","model"]).reset_index(drop=True)

In [127]:
k_list = [10, 20]
metrics = ["P", "R", "NDCG", "HR", "MAP"]

models = {
    "CB": lambda uid, k: preds_cb(uid, k),
    "CF_MF": lambda uid, k: preds_cf_mf(uid, k),
    "HYBRID": lambda uid, k: preds_hybrid(uid, k, alpha=0.3, cf_method="mf"),
}
if "algo_sim" in globals():
    models["CF_ITEM"] = lambda uid, k: preds_cf_item(uid, k)

eval_df = run_evaluation(models, k_list, metrics, test_users, test_truth, train_seen)
display(eval_df)

,model,k,P,P_ci95,R,R_ci95,NDCG,NDCG_ci95,HR,HR_ci95,MAP,MAP_ci95
0,CB,10,0.455440,"[0.434, 0.474]",0.292119,"[0.277, 0.307]",0.585336,"[0.569, 0.603]",0.961252,"[0.945, 0.975]",0.454274,"[0.434, 0.474]"
1,CF_ITEM,10,0.011923,"[0.008, 0.016]",0.005973,"[0.004, 0.008]",0.013198,"[0.009, 0.018]",0.076006,"[0.057, 0.095]",0.005494,"[0.003, 0.008]"
2,CF_MF,10,0.051863,"[0.044, 0.060]",0.028576,"[0.023, 0.035]",0.062965,"[0.053, 0.073]",0.283159,"[0.249, 0.320]",0.031091,"[0.025, 0.038]"
3,HYBRID,10,0.438599,"[0.418, 0.461]",0.269288,"[0.256, 0.284]",0.574201,"[0.554, 0.592]",0.949329,"[0.934, 0.966]",0.445731,"[0.428, 0.464]"
4,CB,20,0.300298,"[0.285, 0.316]",0.337070,"[0.321, 0.352]",0.522098,"[0.505, 0.538]",0.970194,"[0.955, 0.982]",0.367363,"[0.352, 0.382]"
5,CF_ITEM,20,0.012891,"[0.009, 0.017]",0.010860,"[0.007, 0.015]",0.015576,"[0.011, 0.020]",0.116244,"[0.092, 0.142]",0.005610,"[0.004, 0.008]"
6,CF_MF,20,0.039419,"[0.034, 0.045]",0.040098,"[0.034, 0.046]",0.059164,"[0.051, 0.067]",0.357675,"[0.322, 0.391]",0.024332,"[0.019, 0.030]"
7,HYBRID,20,0.284277,"[0.267, 0.303]",0.305256,"[0.289, 0.319]",0.503306,"[0.486, 0.519]",0.962742,"[0.948, 0.976]",0.353267,"[0.337, 0.371]"


In [128]:
print("\n Ablation 1: remove cast features ...")
vectorizer_no_cast = vectorizer.__class__(**vectorizer.get_params())
X_items_no_cast = vectorizer_no_cast.fit_transform(
    movies["overview"].fillna("") + " " +
    movies["tagline"].fillna("") + " " +
    movies["name_genres"].fillna("")
)
user_cb_profiles_no_cast = X_items_no_cast[ [uid_to_index[u] for u in train_seen.keys()] ]

def preds_cb_no_cast(uid, k):
    if uid not in uid_to_index: return []
    u_idx = uid_to_index[uid]
    uvec = user_cb_profiles_no_cast[u_idx,:]
    sims = (uvec @ X_items_no_cast.T).toarray().ravel()
    order = np.argsort(-sims)
    seen = train_seen.get(uid, set())
    rec=[]
    for iidx in order:
        mid = index_item.get(int(iidx))
        if mid is None or mid in seen: continue
        rec.append(mid)
        if len(rec)>=k: break
    return rec


 Ablation 1: remove cast features ...


In [129]:
models_ab1 = {"CB_no_cast": lambda uid,k: preds_cb_no_cast(uid,k)}
eval_ab1 = run_evaluation(models_ab1, k_list, metrics, test_users, test_truth, train_seen)
display(eval_ab1)

,model,k,P,P_ci95,R,R_ci95,NDCG,NDCG_ci95,HR,HR_ci95,MAP,MAP_ci95
0,CB_no_cast,10,0.004471,"[0.003, 0.006]",0.001354,"[0.001, 0.002]",0.005606,"[0.004, 0.008]",0.041729,"[0.027, 0.057]",0.002039,"[0.001, 0.003]"
1,CB_no_cast,20,0.003577,"[0.003, 0.005]",0.002739,"[0.002, 0.004]",0.005153,"[0.003, 0.007]",0.065574,"[0.048, 0.083]",0.001392,"[0.001, 0.002]"


In [130]:
print("\n Ablation 2: CountVectorizer ...")
count_vect = CountVectorizer(max_features=20000, stop_words="english")
X_items_count = count_vect.fit_transform(
    movies["overview"].fillna("") + " " +
    movies["tagline"].fillna("") + " " +
    movies["name_genres"].fillna("")
)
user_cb_profiles_count = X_items_count[ [uid_to_index[u] for u in train_seen.keys()] ]

def preds_cb_count(uid, k):
    if uid not in uid_to_index: return []
    u_idx = uid_to_index[uid]
    uvec = user_cb_profiles_count[u_idx,:]
    sims = (uvec @ X_items_count.T).toarray().ravel()
    order = np.argsort(-sims)
    seen = train_seen.get(uid, set())
    rec=[]
    for iidx in order:
        mid = index_item.get(int(iidx))
        if mid is None or mid in seen: continue
        rec.append(mid)
        if len(rec)>=k: break
    return rec


 Ablation 2: CountVectorizer ...


In [131]:
models_ab2 = {"CB_count": lambda uid,k: preds_cb_count(uid,k)}
eval_ab2 = run_evaluation(models_ab2, k_list, metrics, test_users, test_truth, train_seen)
display(eval_ab2)

,model,k,P,P_ci95,R,R_ci95,NDCG,NDCG_ci95,HR,HR_ci95,MAP,MAP_ci95
0,CB_count,10,0.004769,"[0.003, 0.007]",0.001469,"[0.001, 0.002]",0.005757,"[0.004, 0.008]",0.044709,"[0.030, 0.062]",0.002056,"[0.001, 0.003]"
1,CB_count,20,0.004545,"[0.003, 0.006]",0.002645,"[0.002, 0.004]",0.005605,"[0.004, 0.008]",0.080477,"[0.061, 0.099]",0.001423,"[0.001, 0.002]"


Save Artifacts for App

In [132]:
with open("../models/vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer, f)
with open("../models/X_items.pkl", "wb") as f:
    pickle.dump(X_items, f)
with open("../models/item_index.pkl", "wb") as f:
    pickle.dump(item_index, f)       # tmdbId -> item_idx
with open("../models/index_item.pkl", "wb") as f:
    pickle.dump(index_item, f)       # item_idx -> tmdbId
with open("../models/user_cb_profiles.pkl", "wb") as f:
    pickle.dump(user_cb_profiles, f) # users x features (sparse)

In [133]:
with open("../models/algo_mf.pkl", "wb") as f:
    pickle.dump(algo_mf, f)
if "algo_sim" in globals():
    with open("../models/algo_item_item.pkl", "wb") as f:
        pickle.dump(algo_sim, f)

In [134]:
with open("../models/uid_to_index.pkl", "wb") as f:
    pickle.dump(uid_to_index, f)
with open("../models/train_seen.pkl", "wb") as f:
    pickle.dump(train_seen, f)

In [135]:
movies_meta = movies[["id", "title", "poster_path", "overview", "name_genres"]].copy()
movies_meta["id"] = pd.to_numeric(movies_meta["id"], errors="coerce").astype("Int64")
movies_meta = movies_meta.dropna(subset=["id"]).astype({"id": int}).drop_duplicates("id")

movies_meta_cb = movies_cb[["id","title","overview"]].drop_duplicates("id")
movies_meta_cb = movies_meta_cb.merge(movies_meta[["id","poster_path", "name_genres"]], on="id", how="left")

movies_meta_cb.to_csv("../models/movies_meta.csv", index=False)

In [136]:
pop_candidates.to_csv("../models/pop_candidates.csv", index=False)